In [16]:
# use python 3.6
import time
import datetime
from datetime import date

import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dateutil.relativedelta import relativedelta
# from pandas.tseries.offsets import DateOffset
# from pandas.tseries.offsets import MonthOffset

# finnacial data
from nsepy import get_history
from nsepy.derivatives import get_expiry_date

In [17]:
# path to load and save folders
# create 2 folders - "code" 1 for code and "data_output"
# run notebook from code folder; data will get stored in data_output as csv

cwd = os.getcwd()# ; print(cwd)
save_path = cwd[:-4]# ; print(save_path)

### Story so far

* Previously in black swan post we identified that black swan events are more frequent than a normal distribution would lead us to believe
* We saw the impact of being able to reduce the impact of such events or being able to capture the upside

### Objective

* There are multiple ways of hedging yourself against such events; which can be another post in itself.
* For me the most interesting place to start is using equity options.
* Before we start buying options its always good to backtest different strategies
* In this post I write about downloading 15 years of options data which can be used in a backtest

Note: All the code is based on the documentation of NSEPY(add link) and the book (add link)

### Segue into options 101

* In order to follow further you need to know the basics of options - add youtube video link
* 2 minute Maggi Noodle version 
    * Options are like term insurance - you get a payout if something happens; death in case of term insurance
    * Unlike term insurance - options also have a payout if an event happens 
    * Just like term insurance you need to pay a premium
    * Just like term insurance the arrangement is valid for a predefined period
    * Unlike term insurance options work both ways; i.e. depending on how the contract is made options payoff if the value is above or below the contract price
    * Unlike your life insurance these option contracts can also be bought and sold in the markets; changes in their value depends on multiple factors such as time to expiry, market conditions, prevailing interest ratest, difference between the option contract value and current market price
            * Term Insurance Example - You get paid 10 Lakh rupees if you die anytime during the next 15 years; provided you pay us Rs. 75,000 per year
            * Option Contract Example - You get paid a the market price of the NSE Nifty if it closes above 15,000 on the contract expiry date

Note: This is an extremely simplistic view. Please read and watch some videos to understand this further. And don't trade in options till you don't understand the risks

In [3]:
# lets try to download for a single option first; manually specify dates
sample_opt1 = get_history(symbol = 'NIFTY',
                        start = date(2004,11,1), end = date(2005,1,27),
                        index = True,
                        option_type = 'PE',
                        strike_price = 2000,
                        expiry_date = date(2005,1,27))

print(sample_opt1.shape)
sample_opt1.head(n=3)

(38, 16)


,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2004-12-03,NIFTY,2005-01-27,PE,2000.0,61.95,65.00,57.5,65.00,64.90,65.00,312,128725000.0,NaN,46800,46800,1996.20
2004-12-06,NIFTY,2005-01-27,PE,2000.0,52.10,62.30,52.1,55.40,55.40,55.40,170,70036000.0,NaN,75400,28600,1993.15
2004-12-07,NIFTY,2005-01-27,PE,2000.0,59.00,62.85,59.0,62.85,62.85,62.85,3,1237000.0,NaN,75200,-200,1992.70


###  As seen above

* Downloading manually is too cumbersome since
* multiple strikes
* multiple expiries
* Limits to 1 contract at a time 
* Add screenshot

### Which options data should we download

* Call, Put
* daily, monthly, yearly
* monthly - most liquid
* weekly started around ...
* 3 months prior to expiry
* above and below 1000 points 

### So lets code

* if we have to loop we need to automate the changeover of dates and strike prices; lets 1st run a simple code to do that

In [4]:
# current date - 3 months prior to the 1st option contract expiry
current_date = date(2005, 1,1); print(current_date); print(type(current_date))
type(current_date)

2005-01-01
<class 'datetime.date'>


datetime.date

In [5]:
# price download start date
start_date = current_date + relativedelta(months = -2); print(start_date); print(type(start_date))
start_month = current_date.month; print('Start Month:', start_month)

start_yr = start_date.year; print('Start Year: ', start_yr)


2004-11-01
<class 'datetime.date'>
Start Month: 1
Start Year:  2004


In [6]:
# get expiry date

end_month = current_date.month; print('End Month:', end_month)

end_yr = current_date.year; print('End Year: ', end_yr)

# Use the get expiry function to get a list of expiry dates - sample below
# get_expiry_date returns a list of weekly expiries; use max to get the month end expiry date

expiry_date = max(get_expiry_date(year = end_yr, month = end_month))
print('Expiry_date:', expiry_date, 'Type: ', type(expiry_date))
type(expiry_date)

End Month: 1
End Year:  2005
Expiry_date: 2005-01-27 Type:  <class 'datetime.date'>


datetime.date

In [7]:
# test out for a single option - with using dates as variables before we write the loop
sample_opt2 = get_history(symbol = 'NIFTY',
                        start = start_date, end = expiry_date,
                        index = True,
                        option_type = 'PE',
                        strike_price = 2000,
                        expiry_date = expiry_date)
print(sample_opt2.shape)
sample_opt2.head(n=3)

(38, 16)


,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2004-12-03,NIFTY,2005-01-27,PE,2000.0,61.95,65.00,57.5,65.00,64.90,65.00,312,128725000.0,NaN,46800,46800,1996.20
2004-12-06,NIFTY,2005-01-27,PE,2000.0,52.10,62.30,52.1,55.40,55.40,55.40,170,70036000.0,NaN,75400,28600,1993.15
2004-12-07,NIFTY,2005-01-27,PE,2000.0,59.00,62.85,59.0,62.85,62.85,62.85,3,1237000.0,NaN,75200,-200,1992.70


### Lets loop

* We have to loop over
    * 15 years
    * 12 months in each year
    * At least 6-8 strike prices
    * 2 types of options for each strike
    
* Also the changeover of dates should be random
* We need to download data for 3 months prior to each month to the expiry date of the month
* Start dates have to move in a rolling window
* Give example

In [8]:
# define and month year range to loop over
month_list = np.arange(1, 13, step = 1); print(month_list)
yr_list = np.arange(2005, 2012, step = 1 ); print(yr_list)

[ 1  2  3  4  5  6  7  8  9 10 11 12]
[2005 2006 2007 2008 2009 2010 2011]


In [9]:
# empty dataframe
nifty_data = pd.DataFrame() # to use in the loop
option_data = pd.DataFrame() # to store output


In [10]:
# break the loop into 2 parts to avoid having to rerun everything if there are querying errors
for yr in yr_list:
    # loop through all the months and years
    counter = 0
    print('Year: ', yr)
    for mnth in month_list:
        current_dt = date(yr, mnth, 1)
        start_dt = current_dt + relativedelta(months = -2)
        end_dt = max(get_expiry_date(year = yr, month = mnth))
        
        # print('current: ', current_dt)
        # print('start: ', start_dt)
        # print('end: ', end_dt)
        
        # get nifty futures data
        nifty_fut = get_history(symbol = 'NIFTY',
                               start = start_dt, end = end_dt,
                               index = True,
                               expiry_date = end_dt)
        nifty_data = nifty_data.append(nifty_fut)
        
        # calculate high and low values for each month; round off to get strike prices
        high = nifty_fut['Close'].max()
        high = int(round(high/100)*100) + 1000# ; print('High:', high)
        
        low = nifty_fut['Close'].min()
        low = int(round(low/100)*100) + 1000# ; print('Low :', low)
        
        for strike in range(low, high, 100): # start, stop, step
            """
            get daily closing nifty index option prices for 3 months 
            over the entire range 
            """
            # time.sleep(random.randint(10,20)) # pause for random interval so as to not overwhelm the site
            nifty_opt = get_history(symbol = 'NIFTY',
                                   start = start_dt, end = end_dt,
                                   index = True, option_type = 'PE',
                                   strike_price = strike,
                                   expiry_date = end_dt)
            
            option_data = option_data.append(nifty_opt)
            
            #time.sleep(random.randint(20,50)) # pause for random interval so as to not overwhelm the site
            nifty_opt = get_history(symbol = 'NIFTY',
                                   start = start_dt, end = end_dt,
                                   index = True, option_type = 'CE',
                                   strike_price = strike,
                                   expiry_date = end_dt)
            
            option_data = option_data.append(nifty_opt)
            
        counter+=1
        print('Months: ', counter)
        # print(month)       

Year:  2005
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2006
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2007
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2008
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2009
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2010
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2011
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:

In [11]:
yr_list = np.arange(2012, 2021, step = 1 ); print(yr_list)
#yr_list = np.arange(2018, 2021, step = 1 ); print(yr_list)

[2012 2013 2014 2015 2016 2017 2018 2019 2020]


In [12]:
for yr in yr_list:
    counter = 0
    # loop through all the months and years
    print('Year: ', yr)
    for mnth in month_list:
        current_dt = date(yr, mnth, 1)
        start_dt = current_dt + relativedelta(months = -2)
        end_dt = max(get_expiry_date(year = yr, month = mnth))
        
        # print('current: ', current_dt)
        # print('start: ', start_dt)
        # print('end: ', end_dt)
        
        # get nifty futures data
        nifty_fut = get_history(symbol = 'NIFTY',
                               start = start_dt, end = end_dt,
                               index = True,
                               expiry_date = end_dt)
        nifty_data = nifty_data.append(nifty_fut)
        
        # calculate high and low values for each month; round off to get strike prices
        high = nifty_fut['Close'].max()
        high = int(round(high/100)*100) + 1000# ; print('High:', high)
        
        low = nifty_fut['Close'].min()
        low = int(round(low/100)*100) + 1000# ; print('Low :', low)
        
        for strike in range(low, high, 100): # start, stop, step
            """
            get daily closing nifty index option prices for 3 months 
            over the entire range 
            """
            # time.sleep(random.randint(10,20)) # pause for random interval so as to not overwhelm the site
            nifty_opt = get_history(symbol = 'NIFTY',
                                   start = start_dt, end = end_dt,
                                   index = True, option_type = 'PE',
                                   strike_price = strike,
                                   expiry_date = end_dt)
            
            option_data = option_data.append(nifty_opt)
            
            #time.sleep(random.randint(20,50)) # pause for random interval so as to not overwhelm the site
            nifty_opt = get_history(symbol = 'NIFTY',
                                   start = start_dt, end = end_dt,
                                   index = True, option_type = 'CE',
                                   strike_price = strike,
                                   expiry_date = end_dt)
            
            option_data = option_data.append(nifty_opt)
            
        counter+=1
        print('Months: ', counter)
        # print(month)       

Year:  2012
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2013
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2014
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2015
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2016
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2017
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:  10
Months:  11
Months:  12
Year:  2018
Months:  1
Months:  2
Months:  3
Months:  4
Months:  5
Months:  6
Months:  7
Months:  8
Months:  9
Months:

In [19]:
# visually verify
print(nifty_data.shape)
nifty_data.tail(n=3)

(11589, 6)


,Open,High,Low,Close,Volume,Turnover
Date,,,,,,
2020-12-29,13910.35,13967.60,13859.90,13932.60,439593961,2.515423e+11
2020-12-30,13980.90,13997.00,13864.95,13981.95,380681073,2.463312e+11
2020-12-31,13970.00,14024.85,13936.45,13981.75,452410109,2.630582e+11


In [18]:
# visually verify
print(option_data.shape)
option_data.tail(n=3)

(155846, 16)


,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2020-12-29,NIFTY,2020-12-31,CE,14900.0,1.25,1.35,0.55,0.8,0.80,0.80,1893,2.115557e+09,130000.0,65400,-13725,NaN
2020-12-30,NIFTY,2020-12-31,CE,14900.0,0.75,0.85,0.40,0.7,0.55,0.70,2528,2.825162e+09,122000.0,135975,70575,13981.95
2020-12-31,NIFTY,2020-12-31,CE,14900.0,0.15,0.25,0.05,0.1,0.10,13981.75,4683,5.233288e+09,35000.0,48000,-87975,13981.75


In [15]:
# save to csv for future use
option_data.to_csv(save_path + "data_output/nifty_15 yr_data.csv")